In [6]:
import sys
import os
from prefect import task, Flow

abspath = os.path.dirname(os.path.normpath(os.path.abspath(os.path.dirname(''))))

sys.path.append(abspath)

driver_path = os.path.join(abspath, 'drivers', 'chromedriver.exe')

raw_download_path = os.path.normpath(os.path.join(abspath, 'data', 'raw'))

from toolbox.scrapers import *

# Data Structure: ['HCPCS/ CPT/NDC Code', 'Internal Proc Code', 'Other Code',
#        'Procedure Name', 'Charge', 'Medicare Adv - Aetna',
#        'Medicare Adv - BCBS', 'Medicare Adv - Humana', 'Medicare Adv - United',
#        'Medicare Adv - Other', 'Aetna', 'BCBS', 'Cigna', 'Exchange/Ambetter',
#        'Medcost', 'United', 'Other Mgd Care', 'Tricare', 'Workers Comp',
#        'Self Pay', 'De-identified Minimum', 'De-identified Maximum']


In [7]:
hospital_urls = get_source_urls(driver_path, abspath)


In [8]:
df_ncb = get_north_carolina_baptist('north-carolina-baptist-hospital', hospital_urls, raw_download_path)


In [10]:
df_ncb.columns

Index(['Patient Type', 'DRG', 'NDC', 'Rev Code', 'CPT',
       'Procedure Description', 'Gross Charge', 'Discounted Cash Price',
       'Minimum Negotiated Charge', 'Maximum Negotiated Charge',
       'Aetna Managed Care', 'Aetna Medicare', 'Aetna Whole Health',
       'BCBS\n(PPO, State Health, Federal Employees, Blue Select, Blue Value)',
       'Blue Local Group', 'Blue Local Individual', 'Blue Medicare',
       'Beech Street', 'Carolina Behavioral Health', 'Cigna Behavioral Health',
       'Cigna Healthsprings', 'Cigna Managed Care', 'Coventry Carolinas',
       'Coventry Wellpath', 'First Health', 'HealthTeam Advantage',
       'Humana Choicecare', 'Humana Medicare', 'Medcost', 'Medcost Ultra',
       'Multiplan Private Health Care Systems', 'UHC Behavioral Health',
       'UHC Managed Care', 'UHC Medicare', 'Wellcare Medicare',
       'Inpatient/Outpatient', 'Revenue Code', 'Aetna Wholehealth',
       'BCBS\n(PPO, State Health, Federal Employees, Blue Select)',
       'Blue Value

In [13]:
df_cdm_app

,Code,HCPCS,Billing Description,RevCode,Gross Chg,Self-Pay Discount,Min Allowable,Average Allowable,Max Allowable,Self-Pay,Unnamed: 9
0,611020001,NaN,MED/SURG A PRIVATE ROOM,110.0,987.00,493.50,414.54,651.42,888.30,NaN,NaN
1,611020003,NaN,MED/SURG A HOSPICE ROOM CHARGE,650.0,846.00,423.00,355.32,558.36,761.40,NaN,NaN
2,611020004,G0378,MED/SURG OBSERVATION,762.0,41.50,20.75,17.43,27.39,37.35,NaN,NaN
3,611020005,Q3014,MEDSURG ORIGINATING SITE FEE,780.0,91.00,45.50,38.22,60.06,81.90,NaN,NaN
4,611020006,NaN,IMCU BED CHARGE,206.0,"1,214.00",607.00,509.88,801.24,"1,092.60",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20328,906090013,99442,TELEHEALTH PHONE E/M PHYS/QHP,960.0,67.00,NaN,17.42,37.19,56.95,33.50,NaN
20329,906090014,99443,TELEHEALTH PHONE E/M PHYS/QHP,960.0,99.00,NaN,25.74,54.95,84.15,49.50,NaN
20330,906090015,98966,TELEHEALTH NONPHYSICIAN TELEPH,960.0,33.00,NaN,8.58,18.32,28.05,16.50,NaN
20331,906090016,98967,TELEHEALTH NONPHYSICIAN TELEPH,960.0,67.00,NaN,17.42,37.19,56.95,33.50,NaN


In [15]:
df_cone

,Procedure,Code (CPT/HCPCS/MS-DRG),NDC (for medications/drugs),Rev Code,Procedure Description,Gross Charge,Aetna Negotiated Inpatient Rate,Aetna Negotiated Outpatient Rate,AETNA MEDICARE Negotiated Inpatient Rate,AETNA MEDICARE Negotiated Outpatient Rate,...,VETERANS ADMINISTRATION Negotiated Inpatient Rate,VETERANS ADMINISTRATION Negotiated Outpatient Rate,VIRGINIA PREMIER MEDICARE ADVANTAGE Negotiated Inpatient Rate,VIRGINIA PREMIER MEDICARE ADVANTAGE Negotiated Outpatient Rate,Uninsured Inpatient Rate,Uninsured Outpatient Rate,De-identified Min Inpatient Negotiated Rate Across All Payers,De-identified Max Inpatient Negotiated Rate Across All Payers,De-identified Min Outpatient Negotiated Rate Across All Payers,De-identified Max Outpatient Negotiated Rate Across All Payers
0,MS001,MS001,NaN,NaN,Heart Transplant Or Implant Of Heart Assist Sy...,"$490,771.98","$567,392.99",N/A,"$191,634.16",NaN,...,NaN,N/A,"$191,634.16",NaN,"$215,939.67","$215,939.67","$105,511.76","$745,630.38",N/A,N/A
1,MS002,MS002,NaN,NaN,Heart Transplant Or Implant Of Heart Assist Sy...,"$540,615.96",NaN,N/A,NaN,NaN,...,NaN,N/A,NaN,NaN,"$237,871.02","$237,871.02","$153,683.95","$153,683.95",N/A,N/A
2,MS003,MS003,NaN,NaN,Ecmo Or Tracheostomy With Mv >96 Hours Or Prin...,"$256,535.52","$230,471.89",N/A,"$130,990.95",NaN,...,NaN,N/A,"$128,167.11",NaN,"$112,875.63","$112,875.63","$66,745.36","$303,227.77",N/A,N/A
3,MS004,MS004,NaN,NaN,Tracheostomy With Mv >96 Hours Or Principal Di...,"$256,923.78","$214,397.24",N/A,"$83,339.85",NaN,...,NaN,N/A,"$81,682.00",NaN,"$113,046.46","$113,046.46","$50,880.90","$273,415.48",N/A,N/A
4,MS011,MS011,NaN,NaN,"Tracheostomy For Face, Mouth And Neck Diagnose...","$58,037.92",NaN,N/A,"$36,196.51",NaN,...,NaN,N/A,"$35,477.55",NaN,"$25,536.68","$25,536.68","$34,734.59","$36,196.51",N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113762,ZYG10121,HCPCS C1713,NaN,278.0,DECORTICATORS SIJ SIMMETRY 21,"$2,547.66",NaN,1653.43 to 1933.67,NaN,Not paid separately,...,NaN,N/A,NaN,Not paid separately,"$1,120.97","$1,120.97",Included in DRG Rate,Included in DRG Rate,$428.01,"$2,547.66"
113763,ZYG10121,HCPCS C1713,NaN,270.0,DECORTICATORS SIJ SIMMETRY 21,"$2,547.66",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"$1,120.97","$1,120.97",Included in DRG Rate,Included in DRG Rate,$428.01,$789.77
113764,ZYG10123,NaN,NaN,278.0,DECORTICATORS SIJ SIMMETRY 23,"$2,547.66",NaN,1653.43 to 1933.67,NaN,NaN,...,NaN,N/A,NaN,NaN,"$1,120.97","$1,120.97",Included in DRG Rate,Included in DRG Rate,$428.01,"$2,547.66"
113765,ZYG10123,NaN,NaN,270.0,DECORTICATORS SIJ SIMMETRY 23,"$2,547.66",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"$1,120.97","$1,120.97",Included in DRG Rate,Included in DRG Rate,$428.01,$509.53


In [ ]:
df_cone = get_cone('cone-health', hospital_urls, raw_download_path)

df_cdm_app, df_drg_app, df_shop_app = get_app('app-regional-health-system', hospital_urls, raw_download_path)

df_duke_cdm, df_duke_drg = get_duke('duke-university-hospital', hospital_urls, raw_download_path)

df_catawba = get_catawba('catawba-valley-medical-center', hospital_urls,  raw_download_path)

df_comp_cateret, df_desc_cateret = get_cateret('cateret-health-care', hospital_urls,  raw_download_path)

df_mission = get_mission('mission-health', hospital_urls, raw_download_path)

df_novant = get_novant('novant-health', hospital_urls, raw_download_path)

df_nhrmc_op, df_nhrmc_ip = get_nhrmc('nhrmc-health', hospital_urls, raw_download_path)

df_northern = get_northern('northern-regional', hospital_urls, raw_download_path)

df_first_moore =  get_first('first-health-moore', hospital_urls, raw_download_path)

df_first_montgomery =  get_first('first-health-montgomery', hospital_urls, raw_download_path)

df_cdm_iredell, df_drg_iredell, df_drg_internet =  get_iredell('iredell-health', hospital_urls, raw_download_path)

df_wakemed_cary = get_wakemed('wakemed-cary', hospital_urls, raw_download_path, driver_path)

df_wakemed_raleigh = get_wakemed('wakemed-raleigh', hospital_urls, raw_download_path, driver_path)